In [ ]:
!git clone https://github.com/LC1332/Luotuo-QA.git
!cd ./Luotuo-QA && pip install -r requirements.txt

In [ ]:
from infer import get_model, infer, example_story, example_question

from transformers import AutoTokenizer

model_name: str = "THUDM/chatglm-6b"
peft_path: str = "silk-road/luotuo-qa-lora-0.1"
model_revision: str = "969290547e761b20fdb96b0602b4fd8d863bbb85"
with_origin_model: bool = True

model = get_model(model_name, peft_path)
origin_model = None
if with_origin_model:
    origin_model = get_model(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, revision = model_revision)

In [ ]:
story = example_story
question = example_question

infer(model, tokenizer, story, question, origin_model = origin_model)